In [1]:
from statsforecast.models import SeasonalNaive
import pandas as pd
import numpy as np
from statsforecast.models import SeasonalNaive
from statsforecast import StatsForecast
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

/home/ma/ma_ma/ma_tofuchs/Graph-WaveNet-Mannheim/GraphWave/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load data
timeseries_data = pd.read_csv('/pfs/work9/workspace/scratch/ma_tofuchs-GraphWave-Seminar/Datasets/Mannheim/imputed_dataset_grin_full.csv', index_col=0)

# Compute split indices
num_samples = len(timeseries_data)
train_size = int(num_samples * 0.7)
val_size = int(num_samples * 0.1)

# Split the data
train_data = timeseries_data.iloc[:train_size]
val_data = timeseries_data.iloc[train_size: train_size + val_size]
test_data = timeseries_data.iloc[train_size + val_size:]

# Combine train_data and val_data - remember to uncomment later
train_data = pd.concat([train_data, val_data], ignore_index=False)

# Convert to long format for statsforecast
#train_long = train_data.reset_index().melt(id_vars="index", var_name="sensor_id", value_name="traffic_flow")
train_long.columns = ["ds", "unique_id", "y"]

# Convert test data to long format (needed for forecasting)
test_long = test_data.reset_index().melt(id_vars="index", var_name="sensor_id", value_name="traffic_flow")
test_long.columns = ["ds", "unique_id", "y"]

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['index']"

In [5]:
train_data

,21649702,21649767,21673581,21732938,24555545,25117313,25117359,27088492,27428132,27428349,...,382656460,883463116,988747917,1176328666,1184512491,1633105688,1652156206,1672017001,1946017834,10062245202
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-02-28 23:00:00,658.0,426.0,273.0,149.0,288.0,152.1,126.0,354.0,414.1,171.0,...,123.0,277.0,387.0,480.0,375.0,392.1,551.0,228.0,915.0,438.0
2023-03-01 00:00:00,260.0,450.0,129.0,38.0,303.0,87.1,111.0,186.0,291.0,127.0,...,93.0,148.0,213.0,276.0,141.0,286.9,341.0,222.0,499.0,273.0
2023-03-01 01:00:00,133.8,273.0,72.0,20.0,198.0,67.6,60.0,168.0,127.0,60.0,...,51.0,84.0,180.0,101.0,141.0,127.0,167.0,68.0,221.0,213.0
2023-03-01 02:00:00,107.0,504.0,50.0,31.0,240.0,45.1,69.0,150.0,85.0,38.0,...,39.0,52.0,201.0,57.0,156.0,77.4,126.0,47.0,127.0,369.0
2023-03-01 03:00:00,129.0,417.0,18.0,16.0,288.0,34.0,159.0,297.0,52.0,34.0,...,48.0,57.0,219.0,50.0,243.0,68.6,71.0,46.0,129.0,456.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-16 20:00:00,409.0,3156.0,545.0,869.0,1161.0,354.4,690.0,1458.0,820.1,304.0,...,660.0,500.0,1398.0,642.0,1650.0,915.1,1223.0,508.0,1567.0,1353.0
2023-11-16 21:00:00,640.9,2451.0,361.0,679.6,849.0,255.6,768.0,1230.0,646.0,273.0,...,600.0,431.0,1380.0,555.0,2232.0,797.5,955.0,405.0,1053.5,1431.0
2023-11-16 22:00:00,732.0,2421.0,345.0,260.0,651.0,243.2,378.0,825.0,701.0,667.0,...,285.0,334.0,924.0,406.0,1200.0,1286.1,785.0,325.1,100.0,963.0


In [85]:
# Set seasonality
SEASONALITY = 168

# Initialize statsforecast model
sf = StatsForecast(
    models=[SeasonalNaive(season_length=SEASONALITY)],
    freq="h",  # Adjust based on your dataset (e.g., 'D' for daily, '10T' for 10-minute intervals)
    n_jobs=-1
)

# Fit the model
sf.fit(train_long)

# Forecast for the test set length
forecast_horizon = len(test_data)
forecasts = sf.forecast(df=train_long, h=forecast_horizon)

# Convert back to wide format
forecast_wide = forecasts.pivot(index="ds", columns="unique_id", values="SeasonalNaive")

In [86]:
# Ensure timestamps are in the same format
forecast_wide.index = pd.to_datetime(forecast_wide.index)
test_data.index = pd.to_datetime(test_data.index)

# Force identical index format & align both
test_data = test_data.loc[test_data.index.intersection(forecast_wide.index)]
forecast_wide = forecast_wide.loc[test_data.index]

In [87]:
# Compute per-sensor error metrics
mae_per_sensor = mean_absolute_error(test_data, forecast_wide, multioutput='raw_values')
rmse_per_sensor = np.sqrt(mean_squared_error(test_data, forecast_wide, multioutput='raw_values'))
mape_per_sensor = (np.abs((test_data - forecast_wide) / test_data)).replace([np.inf, -np.inf], np.nan).mean() * 100

# Store per-sensor metrics in a dataframe
sensor_error_metrics = pd.DataFrame({
    "MAE": mae_per_sensor,
    "RMSE": rmse_per_sensor,
    "MAPE (%)": mape_per_sensor
}, index=test_data.columns)

# Compute overall metrics
overall_mae = mae_per_sensor.mean()
overall_rmse = rmse_per_sensor.mean()
overall_mape = mape_per_sensor.mean()

# Store overall metrics in a separate dataframe
overall_error_metrics = pd.DataFrame({
    "MAE": [overall_mae],
    "RMSE": [overall_rmse],
    "MAPE (%)": [overall_mape]
})

In [88]:
sensor_error_metrics

,MAE,RMSE,MAPE (%)
21649702,678.063138,857.223363,86.330277
21649767,1404.037580,1642.041929,38.208792
21673581,2019.235906,2573.955324,24.153979
21732938,444.922264,509.435055,46.735635
24555545,643.993142,850.812612,30.600019
25117313,119.325957,165.432508,31.387494
25117359,607.380809,876.782358,72.997160
27088492,477.509681,627.157719,38.166387
27428132,290.186543,409.002383,27.568542
27428349,2103.699745,2343.174952,57.298174


In [89]:
overall_error_metrics

,MAE,RMSE,MAPE (%)
0,638.448029,819.676823,48.387731
